# Supplemental Analysis for MTA Turnstile Data

Goal:
- Get geographic coordinates (geocodes) of subway locaions 
- data is obtained from census 

### Importing libraries

In [1]:
import pandas as pd
import pickle

### Importing dataset #1  - ZIPCODES
- **subwayszip.csv** is obtained using **MTAData.py**, which uses **Google API** to collect geocodes for NYC subway stations

In [2]:
# Load dataset
station_data = pd.read_csv("../data/subwayszip.csv")

#### Cleaning dataset


In [3]:
# Extracting zipcodes
station_data['zipcode']=station_data.zipcode.str[-6:]
station_data.head(3)

,name,zipcode,lat,long
0,Astor Pl,10003,40.730054,-73.991070
1,Canal St,10013,40.718803,-74.000193
2,50 St Broadway,10019,40.761728,-73.983849


In [4]:
# Converting station names to upper case
station_data['name'] = station_data.name.str.upper()

In [63]:
# Checking unique station names
# station_data.name.unique()

### Importing dataset #2 - INCOME
- **NYC Median Income by Zipcode.csv** is obtained from [Census data for 2017](https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml)

In [5]:
# Importing income data
income_data=pd.read_csv("../data/NYC Median Income by Zipcode.csv")

In [6]:
# Extracting zipcode
income_data['zipcode']=income_data.Geography.str[-6:]

### Merging the two datasets, income and zip codes

In [7]:
station_income=pd.merge(station_data, income_data, 
                        how='left', left_on='zipcode', 
                        right_on='zipcode')

### Sorting by zipcodes

In [8]:
sort_by_zip = station_income.sort_values(by=['lat', 'long'], ascending=[True, True])
sort_by_zip.reset_index(inplace=True)
sort_by_zip.fillna(method='bfill', inplace=True)
sort_by_zip.head(5)

,index,name,zipcode,lat,long,Geography,Median income (dollars); Estimate; Households
0,197,WEST 8 STREET-NEW YORK AQUARIUM,11224,40.576034,-73.975958,ZCTA5 10012,104561
1,99,OCEAN PKWY,11235,40.576312,-73.968501,ZCTA5 10012,104561
2,468,CONEY ISLAND-STILLWELL AV STATION,11224,40.577281,-73.981236,ZCTA5 10012,104561
3,326,BRIGHTON BEACH,11235,40.577710,-73.961354,ZCTA5 10012,104561
4,332,NEPTUNE AV,11224,40.580739,-73.974593,ZCTA5 10012,104561


In [9]:
sort_by_zip['Median income (dollars); Estimate; Households'] = pd.to_numeric(sort_by_zip['Median income (dollars); Estimate; Households'], errors='coerce')

In [10]:
# Collecting median income and zip codes
top_stations = sort_by_zip.sort_values(by=['Median income (dollars); Estimate; Households'], 
                                       ascending=False)
top_stations.to_csv('../data/top_stations.csv')

### Importing Dataset #3 -  MTA data

In [11]:
# Importing 'clean' dataframe from EDA_MTA.ipynb
with open('cleanDf.pickle', 'rb') as readfile:
    dfnew = pickle.load(readfile)

In [12]:
# Creating a subset-df containing only top10 stations
dfStation= dfnew.groupby('Station')[['traffic']].sum().sort_values(by = 'traffic', ascending=False)
list_top10 = dfStation.index[:10].tolist()
df_top10 = dfnew[dfnew.Station.isin(list_top10)]

In [13]:
# Checking the names of the top 10 station names in 'cleaned' MTA data
list_top10

['34 ST-PENN STA',
 'GRD CNTRL-42 ST',
 '34 ST-HERALD SQ',
 '14 ST-UNION SQ',
 'TIMES SQ-42 ST',
 '23 ST',
 'FULTON ST',
 '42 ST-PORT AUTH',
 '86 ST',
 '125 ST']

In [17]:
# Checking unique Station names in df
#sorted(sort_by_zip.name.unique().tolist())

In [18]:
# List of top10 station names in sortedzip df
list_top10_sortedzip = ['34 ST - PENN STATION',
                             'GRAND CENTRAL-42 STREET STATION',
                             '34 ST - HERALD SQ SUBWAY STATION',
                             '14 STREET - UNION SQ STATION',
                             'TIMES SQ-42 ST',
                             '23 STREET STATION',
                             'FULTON ST',
                             '42 ST - PORT AUTHORITY BUS TERMINAL',
                             '86 STREET STATION',
                             '125 ST']

len(list_top10_sortedzip)

10

In [19]:
# Check if the station names from sort_by_zip can be used to merge
for each in sort_by_zip.name:
    if each in list_top10:
        print(each)
        
# (OUTPUT below) Looks like only 3 of themn have same name as the previous ones
# So it needs to be cleaned before merging

FULTON ST
TIMES SQ-42 ST
125 ST


In [20]:
# Create a subset dataframe for top10
df_subsetZip = sort_by_zip.loc[sort_by_zip.name.isin(list_top10_sortedzip),
                               ['name','lat','long']]


In [22]:
# Function to check if the name of stations are consistent, for merging
def checkwork():
    count=0
    for each in df_subsetZip.name.values:
        if each in list_top10:
            count +=1
    if count == 10:
        return 'ready to use!'
    else:
        return "some station names still don't match"
    

checkwork()

"some station names still don't match"

In [23]:
# Let's check the top 10 subset lat/long
df_subsetZip

# (OUTPUT) looks like we have duplicates, 
# because some stations have slightly different lat/long values 

,name,lat,long
41,86 STREET STATION,40.622687,-74.028398
146,FULTON ST,40.687119,-73.975375
261,14 STREET - UNION SQ STATION,40.734673,-73.989951
262,14 STREET - UNION SQ STATION,40.734763,-73.990670
263,14 STREET - UNION SQ STATION,40.735872,-73.990539
269,23 STREET STATION,40.739864,-73.986599
272,23 STREET STATION,40.741303,-73.989344
276,23 STREET STATION,40.742954,-73.992765
280,23 STREET STATION,40.744081,-73.995657
286,23 STREET STATION,40.745906,-73.998041


In [24]:
# Let's keep only one set of lat/long for each station
df_subsetZip.drop([262, 263, 272, 276, 280, 286,
                   301, 307, 311, 360, 361], axis=0, inplace=True)

In [25]:
# Let's check again!
df_subsetZip

# (OUTPUT, below) now we have 10 unique lattitudes and longitudes

,name,lat,long
41,86 STREET STATION,40.622687,-74.028398
146,FULTON ST,40.687119,-73.975375
261,14 STREET - UNION SQ STATION,40.734673,-73.989951
269,23 STREET STATION,40.739864,-73.986599
299,34 ST - HERALD SQ SUBWAY STATION,40.749645,-73.987937
305,GRAND CENTRAL-42 STREET STATION,40.751431,-73.976041
309,34 ST - PENN STATION,40.752287,-73.993391
316,TIMES SQ-42 ST,40.754612,-73.986768
326,42 ST - PORT AUTHORITY BUS TERMINAL,40.757308,-73.989735
385,125 ST,40.815581,-73.958372


In [26]:
# Checking station names in our subsetZip
df_subsetZip.name.values.tolist()

['86 STREET STATION',
 'FULTON ST',
 '14 STREET - UNION SQ STATION',
 '23 STREET STATION',
 '34 ST - HERALD SQ SUBWAY STATION',
 'GRAND CENTRAL-42 STREET STATION',
 '34 ST - PENN STATION',
 'TIMES SQ-42 ST',
 '42 ST - PORT AUTHORITY BUS TERMINAL',
 '125 ST']

In [27]:
# Let's check again
df_subsetZip

,name,lat,long
41,86 STREET STATION,40.622687,-74.028398
146,FULTON ST,40.687119,-73.975375
261,14 STREET - UNION SQ STATION,40.734673,-73.989951
269,23 STREET STATION,40.739864,-73.986599
299,34 ST - HERALD SQ SUBWAY STATION,40.749645,-73.987937
305,GRAND CENTRAL-42 STREET STATION,40.751431,-73.976041
309,34 ST - PENN STATION,40.752287,-73.993391
316,TIMES SQ-42 ST,40.754612,-73.986768
326,42 ST - PORT AUTHORITY BUS TERMINAL,40.757308,-73.989735
385,125 ST,40.815581,-73.958372


In [28]:
# Let's check if we can merge the two station names
checkwork()

"some station names still don't match"

In [29]:
# Function to clean all of the inconsistencies in station names
def cleanstr(df_subsetZip):
    df_subsetZip['name'] = df_subsetZip.name.str.replace('STREET', 'ST')
    df_subsetZip['name'] = df_subsetZip.name.str.replace('STATION', '')
    df_subsetZip.loc[df_subsetZip.name =='34 ST - HERALD SQ SUBWAY','name'] = '34 ST-HERALD SQ'
    df_subsetZip.loc[df_subsetZip.name =='GRAND CENTRAL-42 ST','name'] = 'GRD CNTRL-42 ST'
    df_subsetZip.loc[df_subsetZip.name =='34 ST - PENN','name']= '34 ST-PENN STA'
    df_subsetZip.loc[df_subsetZip.name =='42 ST - PORT AUTHORITY BUS TERMINAL','name'] = '42 ST-PORT AUTH'

    df_subsetZip.loc[df_subsetZip.name =='86 ST ','name'] = '86 ST'
    df_subsetZip.loc[df_subsetZip.name =='14 ST - UNION SQ ','name'] = '14 ST - UNION SQ'
    df_subsetZip.loc[df_subsetZip.name =='23 ST ','name'] = '23 ST'
    df_subsetZip.loc[df_subsetZip.name =='34 ST - HERALD SQ SUBWAY ','name'] = '34 ST - HERALD SQ SUBWAY'
    df_subsetZip.loc[df_subsetZip.name =='GRAND CENTRAL-42 ST ','name'] = 'GRAND CENTRAL-42 ST' 
    df_subsetZip.loc[df_subsetZip.name =='34 ST - PENN ','name'] = '34 ST - PENN'

    df_subsetZip.loc[df_subsetZip.name =='34 ST - PENN','name'] = '34 ST-PENN STA'
    df_subsetZip.loc[df_subsetZip.name =='GRAND CENTRAL-42 ST','name'] = 'GRD CNTRL-42 ST'
    df_subsetZip.loc[df_subsetZip.name =='34 ST - HERALD SQ SUBWAY','name'] = '34 ST-HERALD SQ'
    df_subsetZip.loc[df_subsetZip.name =='14 ST - UNION SQ','name'] = '14 ST-UNION SQ'
    df_subsetZip.loc[df_subsetZip.name =='GRAND CENTRAL-42 ST','name'] = 'GRD CNTRL-42 ST'

    return df_subsetZip

In [30]:
# Use function to clean station names!
cleanstr(df_subsetZip)

,name,lat,long
41,86 ST,40.622687,-74.028398
146,FULTON ST,40.687119,-73.975375
261,14 ST-UNION SQ,40.734673,-73.989951
269,23 ST,40.739864,-73.986599
299,34 ST-HERALD SQ,40.749645,-73.987937
305,GRD CNTRL-42 ST,40.751431,-73.976041
309,34 ST-PENN STA,40.752287,-73.993391
316,TIMES SQ-42 ST,40.754612,-73.986768
326,42 ST-PORT AUTH,40.757308,-73.989735
385,125 ST,40.815581,-73.958372


In [31]:
# Now, check to see if the station names match
checkwork()

'ready to use!'

### Write file 

In [32]:
import pickle

with open('cleanDfsubsetZip.pickle', 'wb') as to_write:
    pickle.dump(df_subsetZip, to_write)

---